## Basic Model Usage

In [14]:
from pydantic import BaseModel, ConfigDict

class User(BaseModel):
    id: int
    name: str = "Jane Doe"

    model_config = ConfigDict(str_max_length = 10)

user = User(id = '123')

In [15]:
# Fields of a model can be accessed as normal attributes of the user object:

assert user.name == "Jane Doe"
assert user.id == 123
assert isinstance(user.id, int)

In [17]:
# The model instance can be serialized using the model_dump() method:

assert user.model_dump() == {'id': 123, 'name': "Jane Doe"}

In [18]:
# By default, models are mutable and field values can be changed through attribute assignment:

user.id = 321
assert user.id == 321

In [19]:
# Validation Error
from typing import Optional

from pydantic import BaseModel

class Boo(BaseModel):
    int: Optional[int] = None

m = Boo(int = 123)

ValidationError: 1 validation error for Boo
int
  Input should be None [type=none_required, input_value=123, input_type=int]
    For further information visit https://errors.pydantic.dev/2.9/v/none_required

## Data Conversion

Pydantic may cast input data to force it to conform to model field types, and in some cases this may result in a loss of information.

In [26]:
from pydantic import BaseModel

class Model(BaseModel):
    a: int
    b: float
    c: str

print(Model(a = 3.000, b = '2.72', c = b'binary data').model_dump())

{'a': 3, 'b': 2.72, 'c': 'binary data'}


In [27]:
# Usage of List

from pydantic import BaseModel

class Model(BaseModel):
    items: list[int]

print(Model(items=(1,2, 3)))

items=[1, 2, 3]


## Extra data
By default, Pydantic models won't error when you provide extra data, and these values will simply be ignored:



In [28]:
from pydantic import BaseModel

class Model(BaseModel):
    x: int

m = Model(x = 1, y = 'a')

assert m.model_dump() == {'x': 1}

In [29]:
from pydantic import BaseModel, ConfigDict

class Model(BaseModel):
    x: int

    model_config = ConfigDict(extra = 'allow')

m = Model(x=1, y = 'a')
assert m.model_dump() == {'x': 1, 'y': 'a'}
assert m.__pydantic_extra__ == {'y': 'a'}

# Nested Models

In [32]:
from pydantic import BaseModel

class Foo(BaseModel):
    count: int
    size: float | None = None

class Bar(BaseModel):
    apple: str = "x"
    banana: str = 'y'

class Spam(BaseModel):
    foo: Foo
    bars: list[Bar]


m = Spam(foo={'count': 4}, bars = [{'apple': 'x1'}, {'apple': 'x2'}])
print(m)

foo=Foo(count=4, size=None) bars=[Bar(apple='x1', banana='y'), Bar(apple='x2', banana='y')]


In [36]:
print(m.model_dump())

{'foo': {'count': 4, 'size': None}, 'bars': [{'apple': 'x1', 'banana': 'y'}, {'apple': 'x2', 'banana': 'y'}]}


# Rebuilding Model Schema


In [40]:
from pydantic import BaseModel, PydanticUserError

class Foo(BaseModel):
    x: 'Bar'

try: 
    Foo.model_json_schema()
except PydanticUserError as e:
    print(e)


class Bar(BaseModel):
    pass

Foo.model_rebuild()
print(Foo.model_json_schema())

{'$defs': {'Bar': {'properties': {}, 'title': 'Bar', 'type': 'object'}}, 'properties': {'x': {'$ref': '#/$defs/Bar'}}, 'required': ['x'], 'title': 'Foo', 'type': 'object'}


# Arbitrary class instances

One common application of this functionality is integration with object-relational mappings (ORMs).

* uses SQLAlchemy

In [45]:
from typing import Annotated

from sqlalchemy import ARRAY, String
from sqlalchemy.orm import DeclarativeBase, Mapped, mapped_column

from pydantic import BaseModel, ConfigDict, StringConstraints


class Base(DeclarativeBase):
    pass

class CompanyOrm(Base):
    __tablename__ = "companies"

    id: Mapped[int] = mapped_column(primary_key = True, nullable=  False)
    public_key: Mapped[str] = mapped_column(
        String(20), index=True, nullable= False, unique=True
    )
    domains: Mapped[list[str]] = mapped_column(ARRAY(String(255)))

class CompanyModel(BaseModel):
    model_config = ConfigDict(from_attributes = True)

    id: int
    public_key: Annotated[str, StringConstraints(max_length=20)]
    domains: list[Annotated[str, StringConstraints(max_length=255)]]


co_orm = CompanyOrm(
    id = 123,
    public_key = 'foobar',
    domains = ['example', 'foobar.com'],
)
print(co_orm)
print()
co_model = CompanyModel.model_validate(co_orm)
print(co_model)


id=123 public_key='foobar' domains=['example', 'foobar.com']


## Nested attributes
When using attributes to parse models, model instances will be created from both top-level attributes and deeper-nested attributes as appropriate.


In [48]:
from pydantic import BaseModel, ConfigDict

class PetCls:
    def __init__(self, *, name: str, species: str):
        self.name = name
        self.species = species

class PersonCls:
    def __init__(self, *, name: str, age: float = None, pets: list[PetCls]):
        self.name = name
        self.age = age
        self.pets = pets

class Pet(BaseModel):
    model_config = ConfigDict(from_attributes = True)

    name: str
    species: str

class Person(BaseModel):
    model_config = ConfigDict(from_attributes = True)

    name: str
    age: float = None
    pets: list[Pet]

bones = PetCls(name = "Bones", species = "dog")
orion = PetCls(name = "Orion", species = 'cat')
anna = PersonCls(name = "Anna", age = 20, pets = [bones, orion])
anna_model = Person.model_validate(anna)

print(anna_model)

name='Anna' age=20.0 pets=[Pet(name='Bones', species='dog'), Pet(name='Orion', species='cat')]


## Error Handling
Pydantic will raise a ValidationError exception whenever it finds an error in the data it's validating.


In [49]:
from pydantic import BaseModel, ValidationError

class Model(BaseModel):
    list_of_ints: list[int]
    a_float: float

data = dict(
    list_of_ints = ['1', 2, 'bad'],
    a_float = 'not a float',
)

try: 
    Model(**data)
except ValidationError as e:
    print(e)

2 validation errors for Model
list_of_ints.2
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='bad', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/int_parsing
a_float
  Input should be a valid number, unable to parse string as a number [type=float_parsing, input_value='not a float', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/float_parsing
